In [1]:
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
import pickle
with open('one_obj_to_rule_them_all.pkl', 'rb') as p:
    obj = pickle.load(p)

data = obj['data']
meta_snowex = obj['meta']['snowex']
meta_awdb = obj['meta']['awdb']
data

,snowex_pit_id,instrument,snowex_id,geometry,swe_snowex,depth_snowex,date,swe_awdb,depth_awdb,awdb_id,distance
2020-01-24,UTLCAC_20200124_1222,None,Alta Collins,POINT (-111.62997 40.57210),436.0,2250.0,2020-01-24,338.0,1270.0,814:UT:SNTL,9979.0
2020-01-24,UTLCAC_20200124_1222,None,Alta Collins,POINT (-111.62997 40.57210),629.0,2190.0,2020-01-24,338.0,1270.0,814:UT:SNTL,9979.0
2020-01-24,UTLCAW_20200124_1100,None,Atwater,POINT (-111.63759 40.59125),436.0,2250.0,2020-01-24,338.0,1270.0,814:UT:SNTL,9528.0
2020-01-24,UTLCAW_20200124_1100,None,Atwater,POINT (-111.63759 40.59125),629.0,2190.0,2020-01-24,338.0,1270.0,814:UT:SNTL,9528.0
2020-01-31,UTLCAC_20200131_1130,None,Alta Collins,POINT (-111.62997 40.57210),732.0,2350.0,2020-01-31,361.0,1295.0,814:UT:SNTL,9979.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-02-14,CAAMCL_20200214_1200,None,Caples Lake,POINT (-120.04185 38.71033),439.0,1230.0,2020-02-14,594.0,NaN,SCN:CA:MSNT,4697.0
2020-02-21,CAAMCL_20200221_1200,None,Caples Lake,POINT (-120.04186 38.71027),421.0,1170.0,2020-02-21,589.0,NaN,SCN:CA:MSNT,4702.0
2020-02-28,CAAMCL_20200228_1130,None,Caples Lake,POINT (-120.04188 38.71025),468.0,1200.0,2020-02-28,587.0,NaN,SCN:CA:MSNT,4703.0
2020-03-06,CAAMCL_20200306_1145,None,Caples Lake,POINT (-120.04187 38.71026),458.0,1130.0,2020-03-06,533.0,NaN,SCN:CA:MSNT,4703.0


In [3]:
# from folium.plugins import MarkerCluster
# points_snowex = MarkerCluster(name='SnowEx measurements')
# points_awdb = MarkerCluster(name='AWDB stations')

meta_awdb['plot_color'] = ['blue' if 'SNTL' in i else 'orange' for i in meta_awdb['site_id']]

map_x = meta_awdb.dissolve().centroid.x[0]
map_y = meta_awdb.dissolve().centroid.y[0]
test_map = folium.Map(location=[map_y, map_x], zoom_start=5, tiles=None, 
                     control_scale=True)
folium.TileLayer('Stamen terrain', name='Terrain basemap', control=True).add_to(test_map)
folium.TileLayer('CartoDB positron', name='Light basemap', control=True).add_to(test_map)

awdb_layer = meta_awdb.explore(
    name='AWDB stations', 
    color='plot_color',
    m=test_map,
    tooltip=['site_name', 'site_id'],
    marker_kwds={'radius': 10}
)

snowex_layer = meta_snowex.explore(
    name='SnowEx metadata', 
    # color='plot_color',
    m=test_map,
    tooltip=['site_name', 'site_id'],
    marker_kwds={'radius': 10}
)

data.index = data.index.astype(str)
data['date'] = data['date'].astype(str)
data_layer = data.explore(
    name='Data',
    # cmap='viridis',
    # color='distance',
    m=test_map,
    # tooltip=[col for col in data.columns if 'geo' not in col],
    marker_kwds={'radius': 10}
)

test_map.add_child(folium.LayerControl())

test_map


# for idx in meta_awdb.index:
#     site_name = meta_awdb.loc[idx, 'site_name']
#     site_id = meta_awdb.loc[idx, 'site_id']
#     folium.Marker(
#         # Need to extract lat/lon from geometry
#         # Marker expects ordered pair, not GeoJSON
#         location=(meta_awdb.loc[idx, 'geometry'].y, meta_awdb.loc[idx, 'geometry'].x), 
#         tooltip=folium.features.Tooltip(
#             text=f'Site name: {site_name}\nSite ID: {site_id}',
#             style=('background-color: white; \
#              color: #333333; \
#              font-family: arial; \
#              font-size: 12px; padding: 10px; white-space: pre')),
#         icon=folium.Icon(
#             color='red', 
#             icon_color='white', 
#             icon='location-pin', angle=0, prefix='fa')).add_to(points_awdb)

# # # Create color column to differentiate points
# # colors = ['red' if el == '36701' else 'green' for el in points['FCode']]
# # points['color'] = colors


# # Create map object
# map_x = meta_awdb.dissolve().centroid.x[0]
# map_y = meta_awdb.dissolve().centroid.y[0]
# huc_map = folium.Map(location=[map_y, map_x], zoom_start=5, tiles=None, 
#                      control_scale=True)
# # Add two background layers
# folium.TileLayer('Stamen terrain', name='Terrain basemap', control=True).add_to(huc_map)
# folium.TileLayer('CartoDB positron', name='Light basemap', control=True).add_to(huc_map)

# # # Create tooltip using HUC8 DataFrame
# # huc_tooltip = folium.GeoJsonTooltip(
# #     fields=['HUC8','Name','AreaSqKm','num_gauges','num_dams'],
# #     aliases=['HUC8: ','Name: ','Area (sq km): ','Number of streamgauges: ','Number of dams: '])

# # # Create a function that returns the style of highlight we want
# # def highlighter(x): 
# #     return {'fillColor': '#000000', 
# #             'color':'#000000', 
# #             'fillOpacity': 0.50, 
# #             'weight': 0.1}

# # # Create a style function
# # # First, create a colormap based on ColorBrewer naming scheme
# # from branca.colormap import linear as linear_cm
# # cmap = linear_cm.YlGnBu_09.scale(min(huc_viz['AreaSqKm']),
# #                                  max(huc_viz['AreaSqKm']))
# # # Style function
# # def styler(x):
# #     return {'fillColor': cmap(x['properties']['AreaSqKm']), 
# #             'color':'#000000', 
# #             'fillOpacity': 1.0, 
# #             'weight': 0.5}

# # # Add HUC8 polygons as GeoJson layer
# # huc_layer = folium.GeoJson(huc_viz, 
# #                            tooltip=huc_tooltip, 
# #                            style_function=styler,
# #                            highlight_function=highlighter,
# #                            name='HUC8')
# # huc_map.add_child(huc_layer)
# # #huc_map.keep_in_front(huc_layer)

# # Add marker cluster layer
# points_awdb.add_to(huc_map)

# # # Add color bar to map
# # cmap.caption = 'Watershed area [sq km]'
# # cmap.add_to(huc_map)

# # Layer control to switch between layers
# huc_map.add_child(folium.LayerControl())
# huc_map

/tmp/ipykernel_698/1177300442.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_x = meta_awdb.dissolve().centroid.x[0]
/tmp/ipykernel_698/1177300442.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_y = meta_awdb.dissolve().centroid.y[0]


In [4]:
data.index

Index(['2020-01-24', '2020-01-24', '2020-01-24', '2020-01-24', '2020-01-31',
       '2020-01-31', '2020-01-31', '2020-01-31', '2020-02-13', '2020-02-13',
       ...
       '2020-03-12', '2020-03-12', '2020-03-12', '2019-12-20', '2020-01-31',
       '2020-02-14', '2020-02-21', '2020-02-28', '2020-03-06', '2020-03-13'],
      dtype='object', length=2598)

In [5]:
for idx in meta_awdb.index:
    print(idx)

155
756
589
197
483
557
636
470
320
660
550
688
307
732
556
877
564
253
98
71
118
33
50
64
114
473
471
472
